In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
df = pd.read_csv('../../Datasets/Lynx_Hare/data.csv', usecols=['YEAR', 'HARE', 'LYNX'])
print(df)

scaler = StandardScaler()
df[['HARE', 'LYNX']] = scaler.fit_transform(df[['HARE', 'LYNX']])

    YEAR   HARE   LYNX
0   1845  19.58  30.09
1   1846  19.60  45.15
2   1847  19.61  49.15
3   1848  11.99  39.52
4   1849  28.04  21.23
..   ...    ...    ...
86  1931  19.52   8.31
87  1932  82.11  16.01
88  1933  89.76  24.82
89  1934  81.66  29.70
90  1935  15.76  35.40

[91 rows x 3 columns]


In [5]:
for col in ['HARE', 'LYNX']:
    df[f'{col}_lag1'] = df[col].shift(1)

df['Lynx_on_Hare'] = df['LYNX_lag1'] * df['HARE']
df.dropna(inplace=True)
print(df)

    YEAR      HARE      LYNX  HARE_lag1  LYNX_lag1  Lynx_on_Hare
1   1846 -0.713379  0.896212  -0.713932   0.093463     -0.066674
2   1847 -0.713103  1.109426  -0.713379   0.896212     -0.639091
3   1848 -0.923746  0.596114  -0.713103   1.109426     -1.024827
4   1849 -0.480069 -0.378806  -0.923746   0.596114     -0.286176
5   1850  0.348128 -1.061622  -0.480069  -0.378806     -0.131873
..   ...       ...       ...        ...        ...           ...
86  1931 -0.715591 -1.067486  -1.138258  -1.138379      0.814614
87  1932  1.014610 -0.657049  -0.715591  -1.067486     -1.083082
88  1933  1.226082 -0.187446   1.014610  -0.657049     -0.805597
89  1934  1.002171  0.072674   1.226082  -0.187446     -0.187853
90  1935 -0.819530  0.376504   1.002171   0.072674     -0.059559

[90 rows x 6 columns]


In [8]:
sequence_length = 5  
X_seqs = []
y_seqs = []

for i in range(len(df) - sequence_length):
    X_seq = df.iloc[i:i + df, :].values
    y_seq = df.iloc[i + df, :].values
    X_seqs.append(X_seq)
    y_seqs.append(y_seq)

X_seq = torch.tensor(np.array(X_seqs), dtype=torch.float32)
y_seq = torch.tensor(np.array(y_seqs), dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

TypeError: cannot do positional indexing on Index with these indexers [    YEAR      HARE      LYNX  HARE_lag1  LYNX_lag1  Lynx_on_Hare
1   1846 -0.713379  0.896212  -0.713932   0.093463     -0.066674
2   1847 -0.713103  1.109426  -0.713379   0.896212     -0.639091
3   1848 -0.923746  0.596114  -0.713103   1.109426     -1.024827
4   1849 -0.480069 -0.378806  -0.923746   0.596114     -0.286176
5   1850  0.348128 -1.061622  -0.480069  -0.378806     -0.131873
..   ...       ...       ...        ...        ...           ...
86  1931 -0.715591 -1.067486  -1.138258  -1.138379      0.814614
87  1932  1.014610 -0.657049  -0.715591  -1.067486     -1.083082
88  1933  1.226082 -0.187446   1.014610  -0.657049     -0.805597
89  1934  1.002171  0.072674   1.226082  -0.187446     -0.187853
90  1935 -0.819530  0.376504   1.002171   0.072674     -0.059559

[90 rows x 6 columns]] of type DataFrame

In [ ]:
class PopulationLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PopulationLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        predictions = self.fc(lstm_out[-1])
        return predictions